In [1]:
!pip install ipython-autotime
%load_ext autotime


time: 0 ns (started: 2022-02-17 11:24:26 +01:00)


In [232]:
pip install torch


time: 2.11 s (started: 2022-02-17 05:25:22 +01:00)



In [39]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.
time: 2.78 s (started: 2022-02-17 02:57:46 +01:00)


In [41]:
pip install shap

Note: you may need to restart the kernel to use updated packages.
time: 3.08 s (started: 2022-02-17 02:58:10 +01:00)


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

time: 0 ns (started: 2022-02-17 11:24:31 +01:00)


In [3]:
import os
import pandas as pd
import numpy as np
import time
import datetime
import itertools
import glob
import re

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
style.use('fivethirtyeight')

from skimage import io

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,Activation,Input,Dropout,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D

from keras.preprocessing.image import ImageDataGenerator


import torch, torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
import numpy as np
import shap

time: 7.92 s (started: 2022-02-17 11:24:33 +01:00)


In [4]:
#Directory path in my com
root_dir ='C:/Users/ANTOINE/Project_Mushroom/' 
raw_data_dir = root_dir + 'raw_data/' #raw images folder
DL_dir = root_dir + 'deep_learning/' #Deep Learning folder

csv_file_paths = 'image_and_json_data.csv'

# export dir with time stamp to save the results of the Deep Learning
timestamp = datetime.datetime.now().strftime('%y%m%d_%HH%M')
export_dir = DL_dir + timestamp + '/'
os.makedirs(export_dir)
print(str(export_dir))

C:/Users/ANTOINE/Project_Mushroom/deep_learning/220217_11H24/
time: 0 ns (started: 2022-02-17 11:24:41 +01:00)


# Variables

In [5]:
family_number = 7
number_of_images = 5000
random_seed = 42
resol = (240, 320)
batch_size = 16

time: 0 ns (started: 2022-02-17 11:24:54 +01:00)


# Dataframe creation and processing

In [8]:
# df with path & target 
df_paths = pd.read_csv(root_dir + csv_file_paths)
print('{} rows in the file'.format(len(df_paths)))
df_paths.head()

208774 rows in the file


,file_name,file_path,resolution,file_year,image_id,format,observation,label,image_url,user,...,gbif_info.canonicalName,gbif_info.phylumKey,gbif_info.class,gbif_info.synonym,gbif_info.scientificName,gbif_info.genus,gbif_info.order,gbif_info.note,gbif_info,json_file
0,1208.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(243, 320)",ages,1208,jpg,1007,Agaricales,http://mushroomobserver.org/images/320/1208,25,...,Agaricales,34.0,Agaricomycetes,False,Agaricales,NaN,Agaricales,NaN,NaN,row_data\json\observations_0.json
1,1604.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 164)",ages,1604,jpg,1535,Agaricales,http://mushroomobserver.org/images/320/1604,43,...,Agaricales,34.0,Agaricomycetes,False,Agaricales,NaN,Agaricales,NaN,NaN,row_data\json\observations_0.json
2,1608.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(320, 286)",ages,1608,jpg,1537,Agaricales,http://mushroomobserver.org/images/320/1608,43,...,Agaricales,34.0,Agaricomycetes,False,Agaricales,NaN,Agaricales,NaN,NaN,row_data\json\observations_0.json
3,29.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(240, 320)",ages,29,jpg,23,Agaricales,http://mushroomobserver.org/images/320/29,1,...,Agaricales,34.0,Agaricomycetes,False,Agaricales,NaN,Agaricales,NaN,NaN,row_data\json\observations_0.json
4,680.jpg,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,"(240, 320)",ages,680,jpg,536,Agaricus,http://mushroomobserver.org/images/320/680,1,...,Agaricus,34.0,Agaricomycetes,False,Agaricus,Agaricus,Agaricales,NaN,NaN,row_data\json\observations_0.json


time: 1.33 s (started: 2022-02-17 11:25:36 +01:00)


In [9]:
#select only 320,240 images with family info
df_paths = df_paths[df_paths['resolution']=='(320, 240)'] 

time: 47 ms (started: 2022-02-17 11:25:41 +01:00)


In [10]:
# keep conf level over 90 
df_paths = df_paths.loc[df_paths['gbif_info.confidence']>90]

time: 47 ms (started: 2022-02-17 11:25:43 +01:00)


In [11]:
# keep only most common class
top_class = df_paths['gbif_info.class'].value_counts().index[0]
df_paths = df_paths[df_paths['gbif_info.class'] == top_class]
df_paths['gbif_info.class'].value_counts()

Agaricomycetes    75164
Name: gbif_info.class, dtype: int64

time: 47 ms (started: 2022-02-17 11:25:46 +01:00)


In [12]:
# drop na 
df_paths = df_paths[df_paths['gbif_info.family'].notna()]

time: 31 ms (started: 2022-02-17 11:25:49 +01:00)


In [13]:
# Check number of classes 
pd.DataFrame(df_paths.groupby(['gbif_info.family'], as_index=False).size())

,gbif_info.family,size
0,Agaricaceae,5090
1,Albatrellaceae,200
2,Amanitaceae,4679
3,Amylocorticiaceae,16
4,Amylostereaceae,6
...,...,...
99,Tubariaceae,212
100,Tulasnellaceae,37
101,Tulostomataceae,2
102,Typhulaceae,39


time: 16 ms (started: 2022-02-17 11:25:51 +01:00)


In [14]:
# keep only top families 

top_fam = df_paths['gbif_info.family'].value_counts().index[:family_number].values
df_paths = df_paths[df_paths['gbif_info.family'].isin(top_fam)]

time: 32 ms (started: 2022-02-17 11:25:56 +01:00)


In [15]:
pd.DataFrame(df_paths.groupby(['gbif_info.family'], as_index=False).size())

,gbif_info.family,size
0,Agaricaceae,5090
1,Amanitaceae,4679
2,Boletaceae,5977
3,Cortinariaceae,3931
4,Russulaceae,5911
5,Strophariaceae,4369
6,Tricholomataceae,4601


time: 15 ms (started: 2022-02-17 11:25:58 +01:00)


In [16]:
# subset col of interest + label encoding
df = df_paths[['file_path','gbif_info.family']].copy()
df['label'] = df['gbif_info.family'].replace(df['gbif_info.family'].unique(), [*range(len(df['gbif_info.family'].unique()))]).astype(str) # label encoding 

df.head()

,file_path,gbif_info.family,label
5,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,Agaricaceae,0
6,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,Agaricaceae,0
7,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,Agaricaceae,0
8,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,Agaricaceae,0
9,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,Agaricaceae,0


time: 47 ms (started: 2022-02-17 11:26:17 +01:00)


In [17]:
df_paths,_ = train_test_split(df_paths,train_size=number_of_images,stratify=df_paths['gbif_info.family'],random_state=random_seed )
print('{} rows in the file'.format(len(df_paths)))

5000 rows in the file
time: 46 ms (started: 2022-02-17 11:26:20 +01:00)


In [18]:
# subset col of interest + label encoding
df = df_paths[['file_path','gbif_info.family']].copy()
df['label'] = df['gbif_info.family'].replace(df['gbif_info.family'].unique(), [*range(len(df['gbif_info.family'].unique()))]).astype(str) # label encoding 

df.head()

,file_path,gbif_info.family,label
123138,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,Russulaceae,0
153128,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,Russulaceae,0
16589,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,Tricholomataceae,1
22151,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,Agaricaceae,2
110231,C:/Users/ANTOINE/Project_Mushroom/row_data\ima...,Strophariaceae,3


time: 0 ns (started: 2022-02-17 11:26:22 +01:00)


# Data for model 

In [20]:
df_train, df_test= train_test_split(df, train_size=0.8, stratify =df['label'], random_state=random_seed)
print(df_train.shape,df_test.shape)

(4000, 3) (1000, 3)
time: 31 ms (started: 2022-02-17 11:27:52 +01:00)


In [21]:
# compute a class weigts dict for unbalanced loss computation
# { cf https://qastack.fr/datascience/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras}
class_weights = class_weight.compute_class_weight(class_weight='balanced',classes =np.unique(df_train['label']),y =df_train['label'])
class_weights = dict(enumerate(class_weights))

time: 0 ns (started: 2022-02-17 11:27:52 +01:00)


In [22]:
# train 
train_data_generator = ImageDataGenerator(
                                          # rotation_range=rotation_range,
                                          # width_shift_range=width_shift_range,
                                          # height_shift_range=height_shift_range,
                                          # shear_range=shear_range,
                                          # zoom_range=zoom_range,
                                          # horizontal_flip=horizontal_flip,
                                          # vertical_flip=vertical_flip
                                          # fill_mode='nearest',
                                        ) 
train_generator = train_data_generator.flow_from_dataframe(dataframe = df_train, 
                                                          x_col = "file_path",y_col="label", 
                                                          shuffle = False,
                                                          target_size=resol,batch_size=batch_size,class_mode='sparse')

#test 
test_generator = ImageDataGenerator().flow_from_dataframe(dataframe = df_test,
                                                        x_col = "file_path",y_col="label", 
                                                          shuffle = False,
                                                          target_size=resol,batch_size=batch_size,class_mode='sparse')



Found 4000 validated image filenames belonging to 7 classes.
Found 1000 validated image filenames belonging to 7 classes.
time: 2.05 s (started: 2022-02-17 11:27:52 +01:00)


# Chargement du modèle

In [36]:
efficientnet = tf.keras.models.load_model(root_dir + 'EFB1_final_model')

time: 15.8 s (started: 2022-02-17 11:38:40 +01:00)


In [37]:
efficientnet.summary()

Model: "EFB1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb1 (Functional)  (None, 8, 10, 1280)      6575239   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                              

In [38]:
efficientnet.layers

time: 0 ns (started: 2022-02-17 11:39:18 +01:00)


In [39]:
test_generator = ImageDataGenerator().flow_from_dataframe(dataframe = df_test,
                                                        x_col = "file_path",y_col="label",
                                                          shuffle = False,
                                                        # to match df_test order 
                                                          target_size=resol,batch_size=batch_size,class_mode='sparse')


Found 1000 validated image filenames belonging to 7 classes.
time: 125 ms (started: 2022-02-17 11:39:20 +01:00)


In [40]:
y_prob = efficientnet.predict(test_generator,len(df_test)//batch_size)
test_pred_class= np.argmax(y_prob , axis=1)

time: 1min 15s (started: 2022-02-17 11:39:23 +01:00)


In [41]:
print(test_pred_class)

[1 3 2 2 1 1 6 1 0 1 0 3 5 2 1 4 5 5 0 4 2 1 5 5 4 6 1 0 3 5 3 3 1 6 4 2 1
 5 5 6 4 0 6 5 2 4 0 0 4 0 0 6 1 2 4 1 4 6 3 1 2 3 0 6 6 4 5 2 2 1 2 4 0 0
 5 1 4 5 2 1 3 6 1 1 2 3 4 0 1 1 0 4 6 0 2 3 0 2 4 2 3 3 0 0 5 0 5 5 3 3 2
 4 6 3 5 6 6 1 4 2 2 6 6 6 0 2 0 1 0 4 5 3 1 4 6 6 3 3 1 5 4 5 4 4 6 3 0 2
 3 4 2 5 4 1 3 3 0 4 6 0 2 6 0 4 2 5 5 0 1 2 0 4 3 5 3 2 3 6 3 2 0 3 3 0 6
 1 4 3 6 6 5 4 5 4 5 6 3 1 0 4 1 6 1 3 6 1 5 0 6 4 0 4 3 6 1 3 3 6 6 6 2 6
 5 6 0 2 6 3 4 1 5 2 3 2 5 0 3 5 3 1 6 1 0 2 4 0 6 0 3 4 4 5 4 1 3 3 0 4 3
 4 0 4 3 6 5 1 3 0 0 2 4 4 4 3 4 3 2 2 4 2 3 5 6 6 2 3 3 0 6 3 1 5 0 3 4 4
 2 2 0 6 3 3 4 1 6 5 3 1 5 2 3 2 3 2 4 3 6 5 2 1 4 0 4 5 1 2 4 4 6 5 2 6 3
 6 3 3 0 0 3 4 3 5 2 0 4 6 0 0 3 3 4 2 5 2 0 2 3 6 4 5 1 0 5 3 3 1 0 2 6 2
 6 3 3 5 6 3 0 3 5 6 5 1 0 0 6 6 1 3 2 3 6 0 2 2 3 4 3 3 2 5 0 5 2 4 4 5 3
 3 0 2 6 3 0 4 6 4 2 6 6 0 2 0 6 1 5 4 3 0 1 2 4 5 6 2 3 6 6 6 0 3 5 6 0 6
 6 1 0 0 3 4 6 3 0 3 6 4 5 1 4 3 4 0 6 2 2 1 1 1 4 2 2 0 6 5 6 2 3 2 1 2 6
 5 3 1 6 2 4 0 3 4 3 6 3 

In [27]:
test_generator.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6}

time: 0 ns (started: 2022-02-17 11:30:16 +01:00)


In [54]:
y_test_class= df_test['label'].values.astype(int)
print(metrics.classification_report(y_test_class, test_pred_class))


              precision    recall  f1-score   support

           0       0.01      0.01      0.01       171
           1       0.03      0.03      0.03       133
           2       0.00      0.00      0.00       147
           3       0.00      0.00      0.00       126
           4       0.00      0.00      0.00       136
           5       0.89      0.88      0.88       114
           6       0.01      0.01      0.01       173

    accuracy                           0.11      1000
   macro avg       0.13      0.13      0.13      1000
weighted avg       0.11      0.11      0.11      1000

time: 0 ns (started: 2022-02-17 11:40:52 +01:00)


# Shap

In [47]:
model = efficientnet

time: 0 ns (started: 2022-02-17 03:07:06 +01:00)


In [87]:
test = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(df_test, batch_size = 110, shuffle = False)


batch = next(iter(test_loader))

NameError: name 'X_test' is not defined

time: 16 ms (started: 2022-02-17 03:26:38 +01:00)


In [56]:
test_generator = ImageDataGenerator().flow_from_dataframe(dataframe = df_test,
                                                        x_col = "file_path",y_col="label", 
                                                          shuffle = False,
                                                          target_size=resol,batch_size=batch_size,class_mode='sparse')


batch = next(iter(test_generator))
images, _ = batch
print(images.shape)
background = images[:10]
test_images= images[10:16]
explainer = shap.DeepExplainer(efficientnet, images)


Found 1000 validated image filenames belonging to 7 classes.
(16, 240, 320, 3)


keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


time: 2.58 s (started: 2022-02-17 12:41:51 +01:00)


In [237]:
test_images

array([[[[ 85.,  77.,  74.],
         [ 80.,  70.,  68.],
         [ 55.,  44.,  42.],
         ...,
         [ 11.,   9.,  10.],
         [ 11.,   9.,  10.],
         [ 10.,   8.,   9.]],

        [[ 68.,  60.,  57.],
         [ 68.,  58.,  56.],
         [ 47.,  36.,  34.],
         ...,
         [  9.,   7.,   8.],
         [ 10.,   8.,   9.],
         [ 11.,   9.,  10.]],

        [[ 60.,  50.,  48.],
         [ 83.,  73.,  71.],
         [ 83.,  72.,  70.],
         ...,
         [ 12.,  10.,  11.],
         [ 15.,  13.,  14.],
         [ 18.,  16.,  17.]],

        ...,

        [[ 79.,  80.,  75.],
         [ 63.,  64.,  59.],
         [ 55.,  56.,  51.],
         ...,
         [ 26.,  17.,  18.],
         [ 16.,   7.,   8.],
         [ 21.,  12.,  13.]],

        [[ 79.,  80.,  75.],
         [ 78.,  79.,  74.],
         [ 67.,  68.,  63.],
         ...,
         [ 28.,  19.,  20.],
         [ 23.,  14.,  15.],
         [ 42.,  32.,  33.]],

        [[ 64.,  65.,  60.],
       

time: 16 ms (started: 2022-02-17 05:26:10 +01:00)


In [57]:
shap_values = explainer.shap_values(images)

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


AttributeError: in user code:

    File "C:\Users\ANTOINE\anaconda3\lib\site-packages\shap\explainers\_deep\deep_tf.py", line 243, in grad_graph  *
        out = self.model(shap_rAnD)
    File "C:\Users\ANTOINE\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ANTOINE\anaconda3\lib\site-packages\shap\explainers\_deep\deep_tf.py", line 26, in custom_record_gradient
        out = tf_backprop._record_gradient("shap_"+op_name, inputs, attrs, results)

    AttributeError: Exception encountered when calling layer "rescaling_1" (type Rescaling).
    
    module 'tensorflow.python.eager.backprop' has no attribute '_record_gradient'
    
    Call arguments received:
      • inputs=tf.Tensor(shape=(32, 240, 320, 3), dtype=float32)


time: 218 ms (started: 2022-02-17 12:41:56 +01:00)


In [77]:
shap_numpy = [np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values]

NameError: name 'shap_values' is not defined

time: 16 ms (started: 2022-02-17 03:21:39 +01:00)


In [62]:
explainer = shap.DeepExplainer(efficientnet,df_train)


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


AttributeError: 'tuple' object has no attribute 'as_list'

time: 16 ms (started: 2022-02-17 03:16:20 +01:00)


In [96]:
for layer in range(7):
    print(efficientnet.layers[layer])
    
    
    

time: 0 ns (started: 2022-02-17 03:32:34 +01:00)


# Test Shap

In [159]:
from keras.preprocessing.image import img_to_array

def get_img_array(img):
    array = img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

# Prepare image
lot_img = next(iter(test_generator))[0]
img = lot_img[0]
#img_array.shapeb


time: 31 ms (started: 2022-02-17 04:46:04 +01:00)


In [162]:
masker = shap.maskers.Image(img, img.shape)


time: 2.42 s (started: 2022-02-17 04:47:52 +01:00)


In [164]:
img.shape

(240, 320, 3)

time: 0 ns (started: 2022-02-17 04:48:11 +01:00)


In [234]:
explainer = shap.Explainer(efficientnet, masker, output_names='famille')


time: 1.27 s (started: 2022-02-17 05:25:46 +01:00)


In [235]:
shap_values = explainer(img[1:3], max_evals=100, batch_size=16, outputs=shap.Explanation)


Exception: The length of the image to be masked must match the shape given in the ImageMasker contructor: 320 * 3 != 240 * 320 * 3

time: 15 ms (started: 2022-02-17 05:25:48 +01:00)


In [167]:
img2, _ = img
e = shap.DeepExplainer(efficientnet,img)

ValueError: too many values to unpack (expected 2)

time: 16 ms (started: 2022-02-17 04:51:36 +01:00)


In [217]:
images, _ = img

ValueError: too many values to unpack (expected 2)

time: 0 ns (started: 2022-02-17 05:19:25 +01:00)


In [216]:
print(test_generator)

time: 0 ns (started: 2022-02-17 05:19:22 +01:00)


In [219]:
print(efficientnet(images))

tf.Tensor(
[[1.7714177e-07 9.9999964e-01 2.3903851e-10 4.7328325e-10 3.5835065e-08
  1.9377333e-07 3.5398692e-10]
 [6.7183962e-08 7.9368689e-09 2.5274496e-09 9.9999392e-01 6.2614568e-08
  7.4697910e-09 5.9049312e-06]
 [1.0106406e-03 1.2401410e-02 5.6906176e-01 7.9432838e-03 7.9705557e-03
  1.6424773e-02 3.8518748e-01]
 [6.8007448e-06 1.7221743e-06 9.8019272e-01 5.7126329e-07 4.5061774e-06
  4.3645206e-05 1.9749960e-02]
 [3.7015723e-06 9.9953198e-01 8.6120463e-06 1.7986417e-07 1.0990243e-05
  4.4279470e-04 1.7852650e-06]
 [4.7032256e-05 9.9089491e-01 4.5072581e-03 4.0401119e-06 9.2530652e-05
  4.4468176e-03 7.4334116e-06]
 [4.9502882e-14 5.3860744e-19 3.4043135e-13 1.6888526e-12 2.4279724e-11
  8.9699324e-15 1.0000000e+00]
 [2.8178529e-06 9.9999607e-01 4.1154666e-09 2.5181749e-11 1.1299085e-08
  1.1152324e-06 5.1284055e-10]
 [9.9999940e-01 1.0253363e-10 6.0569727e-07 1.0121954e-13 2.0717348e-13
  7.4533955e-16 5.7478883e-14]
 [2.5681778e-05 6.6493005e-01 3.3328988e-07 1.1889185e-04 3.12

# Shap training

In [98]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import shap

model = ResNet50(weights='imagenet')
X, y = shap.datasets.imagenet50()

102981632/102967424 [==============================] - 3s 0us/step
time: 5.7 s (started: 2022-02-17 03:46:09 +01:00)


In [142]:
X[0]

array([[[ 33.,  23.,  24.],
        [ 72.,  64.,  61.],
        [ 68.,  61.,  53.],
        ...,
        [ 86., 113., 158.],
        [124., 148., 182.],
        [ 41.,  65., 103.]],

       [[ 63.,  51.,  51.],
        [ 23.,  13.,  11.],
        [ 38.,  29.,  22.],
        ...,
        [ 77., 106., 150.],
        [ 94., 123., 163.],
        [ 72.,  95., 137.]],

       [[ 29.,  20.,  13.],
        [ 37.,  28.,  19.],
        [ 64.,  55.,  46.],
        ...,
        [ 83., 110., 153.],
        [ 94., 127., 170.],
        [131., 153., 194.]],

       ...,

       [[ 98., 153., 122.],
        [ 72., 140., 103.],
        [ 49.,  79.,  53.],
        ...,
        [207., 171.,  75.],
        [ 78.,  61.,  15.],
        [ 56.,  96.,  72.]],

       [[ 92., 142., 113.],
        [ 72., 143., 103.],
        [ 53.,  79.,  52.],
        ...,
        [249., 209., 114.],
        [116.,  91.,  35.],
        [ 62.,  96.,  71.]],

       [[ 96., 151., 122.],
        [ 67., 137., 101.],
        [ 58.,  

time: 16 ms (started: 2022-02-17 04:37:56 +01:00)


In [99]:
# getting ImageNet 1000 class names
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
with open(shap.datasets.cache(url)) as file:
    class_names = [v[1] for v in json.load(file).values()]
#print("Number of ImageNet classes:", len(class_names))
#print("Class nbames:", class_names)

time: 469 ms (started: 2022-02-17 03:46:37 +01:00)


In [199]:
# python function to get model output; replace this function with your own model function.

def f(x):
    tmp = x.copy()
    preprocess_input(tmp)
    return model(tmp)

# define a masker that is used to mask out partitions of the input image.
masker = shap.maskers.Image("inpaint_telea", X[0].shape)

# create an explainer with model and image masker
explainer = shap.Explainer(f, masker, output_names=class_names)

# here we explain two images using 500 evaluations of the underlying model to estimate the SHAP values
shap_values = explainer(X[1:3], max_evals=100, batch_size=50, outputs=shap.Explanation.argsort.flip[:4])

cv2 could not be imported!


ModuleNotFoundError: No module named 'cv2'

time: 46 ms (started: 2022-02-17 05:17:33 +01:00)


SyntaxError: invalid syntax (Temp/ipykernel_8256/3922920591.py, line 1)

time: 0 ns (started: 2022-02-17 05:24:00 +01:00)


# How does this work

In [ ]:
def get_img_array(img):
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

# Prepare image
img_array = get_img_array(img=im)
img_array.shape

In [242]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_data = ImageDataGenerator().flow_from_dataframe(dataframe = df_test,
                                                     x_col = "file_path",y_col="label", 
                                                     shuffle = False, # to match df_test order 
                                                     target_size=(240,320),
                                                     batch_size=16,
                                                     class_mode='sparse')

#assess performances 
start=time.time()
test_pred_class=[]
for i in range(448):
    test_pred_class_temp = efficientnet.predict(test_data[i][0]).argmax(axis = 1) # predictions test
    test_pred_class=np.concatenate([test_pred_class,test_pred_class_temp])
total_time=time.time()
print('durée totale calcul :',total_time)

Found 1000 validated image filenames belonging to 7 classes.


ValueError: Asked to retrieve element 63, but the Sequence has length 63

time: 1min 18s (started: 2022-02-17 05:31:32 +01:00)


In [ ]:
#sélection des indices par class parmi les 7158 images de Test
good_pred_indexes=[]
good_pred_indexes_class0=[]
good_pred_indexes_class1=[]
good_pred_indexes_class2=[]
good_pred_indexes_class3=[]
good_pred_indexes_class4=[]
good_pred_indexes_class5=[]
good_pred_indexes_class6=[]
for i in range(len(test_pred_class['pred_class'])):
    if (test_pred_class['pred_class'].iloc[i]==y_test_class_df['real_class'].iloc[i]):
        good_pred_indexes+=[i]
        if y_test_class_df['real_class'].iloc[i]==0:
            good_pred_indexes_class0+=[i]
        elif y_test_class_df['real_class'].iloc[i]==1:
            good_pred_indexes_class1+=[i]
        elif y_test_class_df['real_class'].iloc[i]==2:
            good_pred_indexes_class2+=[i]
        elif y_test_class_df['real_class'].iloc[i]==3:
            good_pred_indexes_class3+=[i]
        elif y_test_class_df['real_class'].iloc[i]==4:
            good_pred_indexes_class4+=[i]
        elif y_test_class_df['real_class'].iloc[i]==5:
            good_pred_indexes_class5+=[i]
        elif y_test_class_df['real_class'].iloc[i]==6:
            good_pred_indexes_class6+=[i]

In [ ]:
print(np.array(good_pred_indexes_class0).size)
print(np.array(good_pred_indexes_class1).size)
print(np.array(good_pred_indexes_class2).size)
print(np.array(good_pred_indexes_class3).size)
print(np.array(good_pred_indexes_class4).size)
print(np.array(good_pred_indexes_class5).size)
print(np.array(good_pred_indexes_class6).size)
print(np.array(good_pred_indexes).size)

In [ ]:
family_names=test_dict_label['family']
family_names

# retrain model